In [0]:
df=spark.read.csv("/Volumes/workspace/default/ecommercedata/data.csv",header=True,inferSchema=True)


In [0]:
df.head()

Row(InvoiceNo='536365', StockCode='85123A', Description='WHITE HANGING HEART T-LIGHT HOLDER', Quantity=6, InvoiceDate='12/1/2010 8:26', UnitPrice=2.55, CustomerID=17850, Country='United Kingdom')

In [0]:
df.count()

541909

In [0]:
from pyspark.sql.functions import trim,col
df=df.withColumn("Description",trim(col("Description")))

In [0]:
df.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows


## DATA CLEANING

In [0]:
df=df.dropna(subset=["CustomerID"])
df=df.filter(col("Quantity")>0)
df=df.filter(col("UnitPrice")>0)
df=df.filter(col("Description")!="")
df=df.filter(col("Country")!="")
df.count()

397884

**PROBLEM** -> Customer with highest & lowest unit price  

In [0]:
#Highest UnitPrice
df.orderBy(col("UnitPrice").desc()).limit(1).show()



+---------+---------+-----------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+-----------+--------+--------------+---------+----------+--------------+
|   551697|     POST|    POSTAGE|       1|5/3/2011 13:46|  8142.75|     16029|United Kingdom|
+---------+---------+-----------+--------+--------------+---------+----------+--------------+



In [0]:
#Lowest UnitPrice
df.orderBy(col("UnitPrice").asc()).limit(1).show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   550193|     PADS|PADS TO MATCH ALL...|       1|4/15/2011 9:27|    0.001|     13952|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+



**PROBLEM** -> Top 5 countries with highest quantity  

In [0]:
from pyspark.sql.functions import sum
df.groupBy("Country").agg(sum("Quantity").alias("TotalQuantity")).orderBy(col("TotalQuantity").desc()).limit(5).show()

+--------------+-------------+
|       Country|TotalQuantity|
+--------------+-------------+
|United Kingdom|      4256740|
|   Netherlands|       200361|
|          EIRE|       140275|
|       Germany|       119261|
|        France|       111471|
+--------------+-------------+




**PROBLEM** -> Total price of customer (quantity * unitprice)  

In [0]:
df.withColumn("TotalPrice",col("Quantity")*col("UnitPrice")).groupBy("CustomerID").agg(sum("TotalPrice").alias("CustomerSpend")).show(5)

+----------+------------------+
|CustomerID|     CustomerSpend|
+----------+------------------+
|     12451| 9035.519999999999|
|     13011|50.550000000000004|
|     12942| 683.8999999999999|
|     16303| 5360.630000000001|
|     16098|2005.6299999999999|
+----------+------------------+
only showing top 5 rows


**PROBLEM** -> Filter quantity between 4 and 9  

In [0]:
df.filter((col("Quantity")>=4) & (col("Quantity")<=9)).show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows


**PROBLEM** -> Filter customerID = 15862  

In [0]:
df.filter(col("CustomerID")==15862).show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536401|    22110|BIRD HOUSE HOT WA...|       1|12/1/2010 11:21|     2.55|     15862|United Kingdom|
|   536401|    22098|BOUDOIR SQUARE TI...|       1|12/1/2010 11:21|     1.25|     15862|United Kingdom|
|   536401|    22100|SKULLS SQUARE TIS...|       2|12/1/2010 11:21|     1.25|     15862|United Kingdom|
|   536401|    22766| PHOTO FRAME CORNICE|       1|12/1/2010 11:21|     2.95|     15862|United Kingdom|
|   536401|    22451|SILK PURSE BABUSH...|       1|12/1/2010 11:21|     3.35|     15862|United Kingdom|
|   536401|    22549|    PICTURE DOMINOES|       1|12/1/2010 11:21|     1.45|     15862|United Kingdom|
|   536401|    84744|S/6 SEW ON CROCHE...|       1|12/1/2010 11:

In [0]:
display(
  dbutils.fs.ls("/Volumes/workspace/default/ecommercedata")
)

path,name,size,modificationTime
dbfs:/Volumes/workspace/default/ecommercedata/data.csv,data.csv,45580638,1767456830000


In [0]:
output_path="/Volumes/workspace/default/ecommercedata"

In [0]:
dbutils.fs.mkdirs(output_path)


True

In [0]:
df.coalesce(1).write.mode("overwrite").csv(output_path)
display(
  dbutils.fs.ls(output_path)
)

path,name,size,modificationTime
dbfs:/Volumes/workspace/default/ecommercedata/_SUCCESS,_SUCCESS,0,1767519494000
dbfs:/Volumes/workspace/default/ecommercedata/_committed_5623868027042216486,_committed_5623868027042216486,123,1767519494000
dbfs:/Volumes/workspace/default/ecommercedata/_started_5623868027042216486,_started_5623868027042216486,0,1767519491000
dbfs:/Volumes/workspace/default/ecommercedata/part-00000-tid-5623868027042216486-55542e11-e0db-4b4e-8614-0386a13bc5c7-310-1-c000.csv,part-00000-tid-5623868027042216486-55542e11-e0db-4b4e-8614-0386a13bc5c7-310-1-c000.csv,33465913,1767519492000
